# Heating Mesh Tally on CAD geometry made from Shapes

This constructs a reactor geometry from 3 Shape objects each made from points.

The Shapes made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for PF coil

In [ ]:
import paramak

pf_coil = paramak.RotateStraightShape(
    points=[
        (700, 0),
        (750, 0),
        (750, 50),
        (700, 50)
    ],
    stp_filename = 'pf_coil.stp',
    material_tag = 'pf_coil_material'
)

pf_coil.show()

This makes a 3D geometry and material for the centre column

In [ ]:
center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')
    ],
    stp_filename = 'center_column.stp',
    material_tag = 'center_column_material'
)

center_column.show()

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)
    ],
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
    stp_filename = 'blanket.stp',
    material_tag = 'blanket_material'
)

blanket.show()

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, pf_coil,center_column])

my_reactor.solid

At this stage we can export the reactor geometry as stp files and make them avaialbe from download and viewing in FreeCAD.

In [ ]:
my_reactor.export_stp()

from IPython.display import FileLink
display(FileLink('blanket.stp'))
display(FileLink('pf_coil.stp'))
display(FileLink('center_column.stp'))
display(FileLink('Graveyard.stp'))

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:
from neutronics_material_maker import Material

mat1 = Material.from_library(name='Li4SiO4')

mat2 = Material.from_library(name='copper')

mat3 = Material.from_library(name='WC')

This next step makes a simple point source

In [ ]:
import openmc

# initialises a new source object
source = openmc.Source()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:
import paramak_neutronics

neutronics_model = paramak_neutronics.NeutronicsModel(
    geometry=my_reactor,
    mesh_tally_2d=['heating', 'flux', '(n,Xt)'],
    mesh_tally_3d=['heating', 'flux', '(n,Xt)'],
    source=source,
    simulation_batches=10,
    simulation_particles_per_batch=1000,
    materials={
        'blanket_material': mat1,
        'pf_coil_material': mat2,
        'center_column_material': mat3,
    }
)

neutronics_model.simulate()

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_on_3D_mesh.vtk'))
display(FileLink('flux_on_3D_mesh.vtk'))
display(FileLink('tritium_production_on_3D_mesh.vtk'))
display(FileLink('flux_on_2D_mesh_xy.png'))
display(FileLink('flux_on_2D_mesh_xz.png'))
display(FileLink('flux_on_2D_mesh_yz.png'))
display(FileLink('heating_on_2D_mesh_xy.png'))
display(FileLink('heating_on_2D_mesh_xz.png'))
display(FileLink('heating_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_xz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))